#MPS 

# THE MPS CALCULATOR

In [8]:
using Pkg
Pkg.add("ITensors")
Pkg.add("ITensorMPS")
Pkg.update("ITensors")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
    Updating registry at `~/.julia/registries/General.toml`
   Installed ITensors ─ v0.9.14
    Updating `~/.julia/environments/v1.11/Project.toml`
  [9136182c] ↑ ITensors v0.9.8 ⇒ v0.9.14
    Updating `~/.julia/environments/v1.11/Manifest.toml`
  [9136182c] ↑ ITensors v0.9.8 ⇒ v0.9.14
Precompiling project...
  13378.5 ms  ✓ ITensors
   8020.2 ms  ✓ ITensors → ITensorsVectorInterfaceExt
   9780.6 ms  ✓ ITensors → ITensorsTensorOperationsExt
  12691.9 ms  ✓ ITensorMPS
   7410.8 ms  ✓ ITensorMPS → ITensorMPSChainRulesCoreExt
  14718.9 ms  ✓ ITensorVisualizationBase
  12290.2 ms  ✓ ITensorUnicodePlots
  29616.6 ms  ✓ ITensorMakie
  16067.6 ms  ✓ ITensorGLMakie
  9 dependencies successfully 

In [6]:
dir = "/Users/arya/Documents/Python/DATA/"

"/Users/arya/Documents/Python/DATA/"

In [9]:
using LinearAlgebra
using Plots: plot, savefig




function mps_function(psi::Vector{ComplexF64}, cutoff::Float64)
    n = round(Int, log2(length(psi)))  # Number of qubits
    @assert 2^n == length(psi) "Length of psi must be a power of 2"

    mps = []
    D_prev = 1 # Initial bond dimension

    for k in 1:n-1
        d_left = D_prev
        d_phys = 2 # Physical dimension (2 for qubits)
        d_right = length(psi) ÷ (d_left * d_phys)

        psi_matrix = reshape(psi, (d_left * d_phys, d_right))
        U, S, V = svd(psi_matrix)
        
       
      
        if k == ceil(Int, n/2)
            global middle_singular_values = copy(S)
            plt = plot(
                S,
                marker = :circle,
                xlabel = "Number of Singular Values",
                ylabel = "Singular Values",
                title = "Singular Values at Step $k (Middle Site)",
                yscale = :log10,
                ylims = (1e-25, 1e5),
                #yformatter = y -> @sprintf("%.5e", y),   # <--- show 2 decimal digits in scientific notation
                legend = false
            )
            savefig(plt, dir * "singular_values_plot(FGS).pdf")
        end

        

        # Find how many singular values are above cutoff
        r = count(s -> abs(s) > cutoff, S)
        if r == 0
            error("All singular values below cutoff at step $k — increase cutoff or check state")
        end

        # Truncate U, S, V to rank r  (non zero singular values)
        U_trunc = U[:, 1:r]
        S_trunc = S[1:r]
        V_trunc = V[:, 1:r]

        A = reshape(U_trunc, d_left, d_phys, r)
        push!(mps, A)

        psi = Diagonal(S_trunc) * V_trunc'
        D_prev = r
    end

    # Final tensor
    A_last = reshape(psi, D_prev, 2, 1)
    push!(mps, A_last)

    return mps
end



mps_function (generic function with 1 method)

# READING COMPLEX TEXT FILE (2 Functions) 

In [2]:
function load_complex_vector_from_txt(filename::String)
    psi = ComplexF64[]
    open(filename, "r") do file
        for line in eachline(file)
            s = replace(line, "im" => "")  # remove "im"
            parts = split(s, "+")
            if length(parts) == 2
                real_part = parse(Float64, strip(parts[1]))
                imag_part = parse(Float64, strip(parts[2]))
                push!(psi, ComplexF64(real_part, imag_part))
            end
        end
    end
    return psi
end

load_complex_vector_from_txt (generic function with 1 method)

In [3]:
function load_complex_vector_from_txt_2(filename::String)
    psi = ComplexF64[]
    open(filename, "r") do file
        for line in eachline(file)
            # Remove "im" and index number
            parts = split(strip(line), r"\s+")
            if length(parts) >= 4
                real_str = parts[1+1]  # after the index
                sign = parts[2+1]      # '+' or '-'
                imag_str = parts[3+1]

                real_part = parse(Float64, real_str)
                imag_part = parse(Float64, imag_str)
                imag_part *= (sign == "+" ? 1 : -1)

                push!(psi, ComplexF64(real_part, imag_part))
            end
        end
    end
    return psi
end

load_complex_vector_from_txt_2 (generic function with 1 method)

# APLLYING THE MPS CALCULATER ON DIFFERENT STATEVECTORS

In [11]:
psi = load_complex_vector_from_txt(dir * "cl.txt")
MPS = mps_function(psi, 1e-8)
for (i, A) in enumerate(MPS)
    println("A[$i] shape: ", size(A))
end

A[1] shape: (1, 2, 2)
A[2] shape: (2, 2, 4)
A[3] shape: (4, 2, 8)
A[4] shape: (8, 2, 16)
A[5] shape: (16, 2, 32)
A[6] shape: (32, 2, 16)
A[7] shape: (16, 2, 8)
A[8] shape: (8, 2, 4)
A[9] shape: (4, 2, 2)
A[10] shape: (2, 2, 1)


## Renyi Entropy

In [12]:
function renyi_entropies(S::AbstractVector)
    p = S.^2
    p ./= sum(p)
    Ren_En = zeros(Float64, 10)
    #Van Neumann Entropy
    Ren_En[1] = -sum(pi == 0 ? 0.0 : pi * log(pi) for pi in p)          #Note to myself: Ternary Operator
    #Renyi Entropies
    for n in 2:10
        Ren_En[n] = (1 / (1 - n)) * log(sum(p .^ n))
    end

    return Ren_En
end

renyi_entropies (generic function with 1 method)

#### Checking the singular values

In [13]:
middle_singular_values

32-element Vector{Float64}:
 0.1767760000000001
 0.1767760000000001
 0.1767760000000001
 0.17677600000000007
 0.17677600000000007
 0.17677600000000007
 0.17677600000000007
 0.17677600000000007
 0.17677600000000007
 0.17677600000000007
 ⋮
 0.176776
 0.176776
 0.176776
 0.176776
 0.17677599999999996
 0.17677599999999996
 0.17677599999999996
 0.17677599999999993
 0.1767759999999999

#### Renyi Entropy of Haar random (2-10)

In [14]:
E1 = renyi_entropies(middle_singular_values)

10-element Vector{Float64}:
 3.465735902799726
 3.4657359027997257
 3.4657359027997257
 3.4657359027997257
 3.465735902799726
 3.465735902799726
 3.4657359027997257
 3.4657359027997257
 3.4657359027997257
 3.4657359027997257

#### Renyi Entropy of Stablizer (2-10)

In [15]:
#Clifford_10_4
E2 = renyi_entropies(middle_singular_values)


10-element Vector{Float64}:
 3.465735902799726
 3.4657359027997257
 3.4657359027997257
 3.4657359027997257
 3.465735902799726
 3.465735902799726
 3.4657359027997257
 3.4657359027997257
 3.4657359027997257
 3.4657359027997257

# SERs

### Building the Pauli Strings

In [2]:
labels = readlines("/Users/arya/Documents/Python/DATA/P_labels.txt")

1024-element Vector{String}:
 "IIIIIIIIII"
 "-XYZIYXIIIY"
 "-XIXYYIYXXZ"
 "-IYYYIXYXXX"
 "XXZIXIIZYI"
 "IZIIZXIZYY"
 "IXYYZIYYZZ"
 "-XZXYXXYYZX"
 "XIYZYXYXXZ"
 "-IYXZIIYXXX"
 ⋮
 "XZXIIYYXXZ"
 "-XIYXZYYYZX"
 "IYXXXZYYZZ"
 "-IIZZXYIZYY"
 "XYIZZZIZYI"
 "IXXXYYYXXX"
 "-XZYXIZYXXZ"
 "-XXIZIYIIIY"
 "IZZZYZIIII"

In [8]:
using LinearAlgebra

#defining Pauli Matrices
s_x = ComplexF64[0 1; 1 0]
s_y = ComplexF64[0 -im; im 0]
s_z = ComplexF64[1 0; 0 -1]
s_i = Matrix{ComplexF64}(I, 2, 2)

# Map labels to matrices
if !@isdefined pauli_map
    const pauli_map = Dict{Char, Matrix{ComplexF64}}('X'=>s_x, 'Y'=>s_y, 'Z'=>s_z, 'I'=>s_i)  
end 


function pauli_to_mpo(label::String)
    s = strip(label)
    coef = startswith(s, '-') ? (s = s[2:end]; -1.0) : 1.0
    mpo = [reshape(pauli_map[c], 1,1,2,2) for c in s]  # convert Char→String
    mpo[1] .*= coef
    return mpo
end



pauli_to_mpo (generic function with 1 method)

In [9]:
mpo = pauli_to_mpo(labels[1])

10-element Vector{Array{ComplexF64, 4}}:
 [1.0 + 0.0im;;; 0.0 + 0.0im;;;; 0.0 + 0.0im;;; 1.0 + 0.0im]
 [1.0 + 0.0im;;; 0.0 + 0.0im;;;; 0.0 + 0.0im;;; 1.0 + 0.0im]
 [1.0 + 0.0im;;; 0.0 + 0.0im;;;; 0.0 + 0.0im;;; 1.0 + 0.0im]
 [1.0 + 0.0im;;; 0.0 + 0.0im;;;; 0.0 + 0.0im;;; 1.0 + 0.0im]
 [1.0 + 0.0im;;; 0.0 + 0.0im;;;; 0.0 + 0.0im;;; 1.0 + 0.0im]
 [1.0 + 0.0im;;; 0.0 + 0.0im;;;; 0.0 + 0.0im;;; 1.0 + 0.0im]
 [1.0 + 0.0im;;; 0.0 + 0.0im;;;; 0.0 + 0.0im;;; 1.0 + 0.0im]
 [1.0 + 0.0im;;; 0.0 + 0.0im;;;; 0.0 + 0.0im;;; 1.0 + 0.0im]
 [1.0 + 0.0im;;; 0.0 + 0.0im;;;; 0.0 + 0.0im;;; 1.0 + 0.0im]
 [1.0 + 0.0im;;; 0.0 + 0.0im;;;; 0.0 + 0.0im;;; 1.0 + 0.0im]

In [47]:
Pauil_MPOs = []
for label in labels
    mpo = pauli_to_mpo(label)
    push!(Pauil_MPOs, mpo)
end

InexactError: InexactError: Bool(-1.0)

# RECONSTRUCTING  THE STATEVECTOR BY CONTRACTING MPS ELEMENTS

In [2]:
using TensorOperations

function contract_mps(MPS::Vector)
    psi = MPS[1]  # Start with the first site tensor
0
    for i in 2:length(MPS)
        T = MPS[i]

        
        @tensor psi_new[a, p1, p2, c] := psi[a, p1, b] * T[b, p2, c]

        
        d1 = size(psi, 2)
        d2 = size(T, 2)
        right_bond = size(psi_new, 4)

        psi = reshape(psi_new, 1, d1 * d2, right_bond)
    end

    
    return reshape(psi, :)
end

contract_mps (generic function with 1 method)

# CUTTING OFF THE COMPLEX NUMBERS 

In [1]:
function cutoff_complex_parts(A::Array, cutoff::Float64 = 1e-12)
    re = real.(A)
    im = imag.(A)

    re[abs.(re) .< cutoff] .= 0.0
    im[abs.(im) .< cutoff] .= 0.0

    A .= complex.(re, im)
end

cutoff_complex_parts (generic function with 2 methods)

# APPLYING THE RECONSTRUCTING FUNCTION

In [35]:
re_psi_1 = contract_mps(MPS)
re_psi = cutoff_complex_parts(re_psi_1, 1e-8)



4096-element Vector{ComplexF64}:
    0.02221278104648671 + 0.0im
                    0.0 + 0.0im
                    0.0 + 0.0im
   0.006459266754389635 - 0.013653612995951827im
                    0.0 + 0.0im
   -0.01564549559844753 - 0.010153208000613503im
   0.014734130944164855 + 0.011194121942464162im
                    0.0 + 0.0im
                    0.0 + 0.0im
 -0.0003580795991152047 + 0.009868844424071804im
                        ⋮
                    0.0 + 0.0im
                    0.0 + 0.0im
   0.005425202135683498 - 0.006361943170859172im
   0.002326104433596305 - 0.0015250884178343662im
                    0.0 + 0.0im
  -0.011019829568911775 + 0.0109105172658617im
                    0.0 + 0.0im
                    0.0 + 0.0im
    0.00718388943285008 + 0.0im

# CHECKING THE RECONSTRUCTED STATEVECTOR IS SAME AS INITIAL ONE

In [36]:
norm(re_psi - psi)

1.6981183086019288e-7